# Temp

To delete.

## Setup

In [ ]:
# yolo
import warnings

warnings.simplefilter("ignore")

## Installs

In [ ]:
%pip install rapidfuzz

In [ ]:
#%pip install --upgrade 'snowflake-snowpark-python[pandas]' pyyaml numpy pandas scikit-learn matplotlib seaborn prophet mlflow thefuzz ipympl

## Imports

In [ ]:
import yaml
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from random import random, randint
from thefuzz import fuzz
from prophet import Prophet
from datetime import datetime
from sklearn.cluster import KMeans
from prophet.serialize import model_to_json, model_from_json
from snowflake.snowpark import Session
from sklearn.decomposition import PCA

from thefuzz.process import extractOne as match_str

## Connect to Snowpark in Python

In [ ]:
# replace as needed
PROFILE_PATH = "/root/.dbt/profiles.yml"
PROFILE_NAME = "snowflake"
PROFILE_OUTPUT = "dev"

# read in dbt profile
with open(PROFILE_PATH, "r") as f:
    profiles = yaml.safe_load(f)
    profile = profiles[PROFILE_NAME]["outputs"][PROFILE_OUTPUT]

# build connection parameters from profile
conn_params = {
    "account": profile["account"],
    "user": profile["user"],
    "role": profile["role"],
    "warehouse": profile["warehouse"],
    "database": profile["database"],
    "schema": profile["schema"],
    "authenticator": profile["authenticator"],
}
conn_params

In [ ]:
schema = conn_params["schema"]

In [ ]:
from snowflake.snowpark import Session

s = Session.builder.configs(conn_params).create()
s

In [ ]:
s.sql("select current_warehouse(), current_database(), current_schema()").collect()

## Fun stuff

In [ ]:
int_customers = s.table(f"{schema}.int_customers").to_pandas()
int_customers.head()

In [ ]:
int_orders = s.table(f"{schema}.int_orders").to_pandas()
int_orders.tail()

In [ ]:
from thefuzz.process import extractOne

In [ ]:
names = int_customers["customer_name".upper()].unique()
names

In [ ]:
from rapidfuzz.process import extractOne as match_str

In [ ]:
match_str("Brenda", names)

In [ ]:
%%time

sample = int_orders.sample(4000)

int_orders[["matched_name".upper(), "matched_likelihood".upper()]] = sample.apply(
    lambda x: match_str(x["customer_name".upper()], names)[0:2], axis=1, result_type="expand"
)
int_orders.iloc[sample.index].head()

In [ ]:
import dask

from dask import delayed
from distributed import Client

In [ ]:
c = Client()
c

In [ ]:
def process_batch(batch):

    # use this but with column names as matched_name, matched_likelihood
    return batch.apply(
        lambda x: match_str(x["customer_name".upper()], names),
        axis=1,
        result_type="expand",
    )

In [ ]:
results = []
for df in np.array_split(int_orders, 1000):
    results.append(delayed(process_batch)(df))
results[0:3]

In [ ]:
dask.compute(*results)

In [ ]:
dask.compute(*results)

In [ ]:
%%time

test = int_orders.sample(20).apply(
    lambda x: match_str(x["customer_name".upper()], names), axis=1, result_type="expand"
)
test

In [ ]:
type(test)